# Azure Machine Learning による機械学習プロセス - モデル学習編
本ノートブックでは、オープンソースライブラリの LightGBM (Python API) を用いてモデル学習を行っていきます。各機械学習プロセスを Azure Machine Learning Studio and/or Azure Machine Learning Python SDK を用いて管理します。

## アジェンダ
### **A. 学習編 (本ノートブック)**
- ワークスペース (Workspace) への接続
- データセット (Datasets) の登録
- 環境 (Environments) の登録
- コンピューティングクラスター (Compute Clusters) の作成
- モデル学習の実行と実験 (Run & Experiments)
- モデル登録 (Models)

### B. デプロイ編
- 推論環境の作成 (Deployments)
- エンドポイントの利用 (Endpoint)

## 事前設定
- 本ノートブックは Azure Machine Learning の Compute Instance を利用することを想定しています。
- 開発環境は JupyterLab, VSCode, Integrated Notebook など Compute Instance で稼働するものであれば自由に選択いただけます。
- カーネルは `python38-azureml (Python 3.8 AzureML)` を選択ください。

<div class="alert alert-info"><h5> 本ノートブックの進め方</h5><p>
基本的には Azure Machine Learning Python SDK を用いて作業を進めてください。参考情報として Azure Machine Learning Studio での実行手順も記載しています。</p></div>

<br>

## ワークスペース (Workspace) への接続
作業環境から Azure Machine Learning Workspace へ接続を行います。

#### Python SDK での手順
クライアント環境の Python 環境にインストールした Azure ML Python SDK を用いて Azure Machine Learning Workspace に接続します。

In [ ]:
# Compute Instance を利用する場合
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
# # その他の任意のクライアント環境を利用する場合
# ws = Workspace.get(
#     name='name',
#     subscription_id='subscription_id',
#     resource_group='resource_group',
# )

#### (参考) Azure Machine Learning Studio での手順
[ml.azureml.com](ml.azurem.com) にアクセスします。Python SDK を中心に作業される場合にも Azure Machine Learning Studio を併用することが多いため、この画面は開いておきます。

<img src="../docs/images/azureml-workspace.png" width=500>


<br>

## データセット (Datasets) の登録
Azure のストレージやデータベースに格納されているデータをデータセット (Datasets) として登録します。学習データになります。

#### Python SDK での手順
データソースへの接続情報を保持しているデータストア (Datastores) を利用して、CSV ファイルを Azure ストレージ (Azure ML のデフォルトストレージ) にアップロードします。その後、そのファイルをデータセット (Datasets) として登録します。

In [ ]:
from azureml.core import Dataset

# データストア (Datastore) へのアップロード
datastore = ws.get_default_datastore()
datastore.upload_files(files=['../data/Titanic.csv'],
                 target_path='demo',
                 overwrite=True)

# データセット (Dataset) の登録
datastore_paths = [(datastore, 'demo/Titanic.csv')]
# 表形式を選択
titanic_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)
titanic_ds.register(ws, "titanic", create_new_version=True)

Azure Machine Learning Studio にて正常に登録されていることを確認します。<br>
<img src="../docs/images/azureml-dataset4.png" width=900><br>



#### (参考) Azure Machine Learning Studio での手順
ブラウザを開いている作業端末に CSV (data フォルダの Titanic.csv) をダウンロードして、データセット (Dataset) として登録します。

データセットの名称は `titanic` として、データセットの種類は _表形式_ とします。<br>
<img src="../docs/images/azureml-dataset1.png" width=500><br>
data フォルダの Titanic.csv データをアップロードします。次にファイル形式、区切り記号などの情報に誤りがないことを確認して次に進みます。<br>
<img src="../docs/images/azureml-dataset2.png" width=500><br>
スキーマの設定に誤りがないことを確認して次に進み、データセットの登録を完了します。<br>
<img src="../docs/images/azureml-dataset3.png" width=500><br>
正常に登録されていることを確認します。<br>
<img src="../docs/images/azureml-dataset4.png" width=900><br>


<br>

## 環境 (Environment) の登録
モデル学習や推論で利用する Python ライブラリとそのバージョンを環境 (Environment) として登録します。

#### Python SDK での手順
環境の名称は `lightgbm-python-env` とし、「Python 仮想環境」を選択し、Python ライブラリとそのバージョンが記載されている [environments/requirements.txt](environments/requirements.txt) のファイルを読み込み、環境を作成します。

In [ ]:
from azureml.core import Environment
environment_name = "lightgbm-python-env"
file_path = "../environments/requirements.txt"
env = Environment.from_pip_requirements(name = environment_name, file_path = file_path)
env.register(ws)

Azure Machine Learning Studio にて正常に登録されていることを確認します。<br>
<img src="../docs/images/azureml-environment2.png" width=500><br>

#### (参考) Azure Machine Learning Studio での手順

環境の名称は `lightgbm-python-env` とし、「Python 仮想環境」を選択し、Python ライブラリとそのバージョンが記載されている [environments/requirements.txt](environments/requirements.txt) の内容をコピー&ペーストします。<br>
次へ進み、内容に誤りない確認をして環境を作成します。<br>
<img src="../docs/images/azureml-environment1.png" width=500><br>
正常に環境が登録されていることを確認します。<br>
<img src="../docs/images/azureml-environment2.png" width=500><br>

<br>

## コンピューティングクラスター (Compute Clusters) の作成
(バッチ的な) モデル学習に利用する計算クラスターであるコンピューティングクラスター (Compute Clusters) を作成します。

#### Python SDK での手順
名称は `cpu-clusters` とし、(例えば) Standard_F4s_v2 VM ファミリーを選択します。<br>
最小ノード数は 0、最大ノード数は 4 などに設定し、コンピューティングクラスターを作成します。<br>

In [ ]:
compute_name = "cpu-clusters"

(注意) この Compute Clusters は Azure Machine Learning Workspace で共有で利用されるものです。複人で作業をしている場合は、誰か一人が作成するだけで十分です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

#TODO: 仮想ネットワークの情報を記載
# vnet_resourcegroup_name = ""
# vnet_name = ""
# subnet_name = "default"

if compute_name not in ws.compute_targets: # compute_name の名前の Compute Cluster が無ければ...
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_F4S_V2", 
                                                           max_nodes=4, 
                                                           idle_seconds_before_scaledown = 300,
                                                           #vnet_resourcegroup_name=vnet_resourcegroup_name,
                                                           #vnet_name=vnet_name,
                                                           #subnet_name=subnet_name
                                                           )

    ct = ComputeTarget.create(ws, compute_name, compute_config)
    ct.wait_for_completion(show_output=True)

Azure Machine Learning Studio にて正常に登録されていることを確認します。<br>
<img src="../docs/images/azureml-computeclusters4.png" width=500><br>



#### (参考) Azure Machine Learning Studio での手順

「新規」からコンピューティングクラスターを作成していきます。<br>
<img src="../docs/images/azureml-computeclusters1.png" width=500><br>

(例えば) Standard_F4s_v2 などの VM ファミリーを選択します。<br>
<img src="../docs/images/azureml-computeclusters2.png" width=500><br>

名称は `cpu-clusters` とし、最小ノード数は 0、最大ノード数は 4 などに設定し、コンピューティングクラスターを作成します。<br>
<img src="../docs/images/azureml-computeclusters3.png" width=500><br>

正常に作成されたことを確認します。<br>
<img src="../docs/images/azureml-computeclusters4.png" width=500><br>


<br>

## モデル学習の実行と実験 (Run & Experiments)
モデル学習を実行します。それぞれの実行 (Run) は実験 (Experiments) に情報が集約されます。

#### Python SDK での手順
実験名を `lgb-train` とします。

In [ ]:
from azureml.core import Experiment

experiment_name = "lgb-train"
experiment = Experiment(ws, experiment_name)

ScriptRunConfig で実行の構成設定をします。
- 参考ドキュメント : [トレーニングの実行を構成して送信する - スクリプト実行構成とは](https://docs.microsoft.com/ja-JP/azure/machine-learning/how-to-set-up-training-targets#whats-a-script-run-configuration)

なお、学習コードは [src/train-lgb.py](src/train-lgb.py) から確認できます。学習データは Azure Machine Learning に登録されているものを呼びだしています。メトリックの記録は MLflow の Autolog 機能などを用いています。

In [ ]:
from azureml.core import ScriptRunConfig

script_dir = "script"
script_name = "train-lgb.py"
args = ["--input-data", titanic_ds.as_named_input('titanic')]

src = ScriptRunConfig(
    source_directory=script_dir,
    script=script_name,
    environment=env,
    arguments=args,
    compute_target=compute_name,
)

ジョブを実行します。

In [ ]:
run = experiment.submit(src)
run

ジョブ実行が完了するまで待機します。

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
# run.download_files(output_directory=experiment_name+"_logging")

In [ ]:
run

Azure Machine Learning Studio にて正常にジョブが実行されたことを確認します。<br>
<img src="../docs/images/azureml-experiment1.png" width=500><br>
実行時間、メトリック、コードなどの情報が確認できます。<br>
<img src="../docs/images/azureml-experiment2.png" width=1500><br>


#### (参考) Azure Machine Learning Studio での手順


<div class="alert alert-info"><h4> 注意 </h4><p>
Azure Machine Learning から Job を実行する機能はプレビューです。<br>
正式にサポートされていない (2022年1月時点) ことと、表形式のデータセット (Dataset) がまだサポートされていないので、ここでは割愛します。</p></div>

<br>

## モデル登録 (Model Registry)
学習済みモデルファイルや関連ファイルを登録します。

#### Python SDK での手順
ジョブ実行の run オブジェクトを利用して、モデル登録をします。<br>
※ Azure Machine Learning Studio 経由で実行する場合と違って、クライアント端末への学習済みモデルのダウンロードは不要です。

In [ ]:
from azureml.core import Model

model = run.register_model(
    model_name="lgb-model",
    model_framework="LightGBM",
    model_framework_version="3.3.1",
    tags={'algorithm': 'lightGBM'}, 
    model_path = 'model')

Azure Machine Learning Studio にて正常に登録されていることを確認します。<br>
<img src="../docs/images/azureml-model2.png" width=500><br>

#### (参考) Azure Machine Learning Studio での手順
ブラウザを開いている作業端末に学習済みモデルなどのファイルが含まれる _model_ フォルダをダウンロードしておきます。

名称は `lgb-model` とします。利用してフレームワークである LightGBM やバージョンを記載します。また _model_ フォルダごとアップロードして、モデルとして登録します。<br>
<img src="../docs/images/azureml-model1.png" width=500><br>

登録された内容を確認します。<br>
<img src="../docs/images/azureml-model2.png" width=500><br>
